In [5]:
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import MarkdownTextSplitter,RecursiveCharacterTextSplitter
from langchain.prompts import ChatPromptTemplate

from langchain_community.llms import Ollama

from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
import tqdm
import os

from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


In [2]:
from langchain.document_loaders import PyPDFLoader
#from langchain_community.document_loaders import WebBaseLoader

loader = PyPDFLoader("docs/Sách Deep Learning cơ bản.pdf")
# loader = WebBaseLoader("https://docs.smith.langchain.com/user_guide")

pages = loader.load()

In [34]:

# text_splitter = MarkdownTextSplitter(
#     chunk_size=1000,
#     chunk_overlap=200,
# )
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)


In [37]:
docs = text_splitter.split_documents(pages)
for idx, text in enumerate(docs):
    text.metadata["id"] = idx

In [39]:
print(docs[1])

page_content='3\nLời mở đầu\nGiới thiệu về Deep Learning\nNhững năm gần đây, AI - Artificial Intelligence (Trí Tuệ Nhân Tạo), và cụ thể hơn là Machine\nLearning (Máy Học) nổi lên như một minh chứng của cuộc cách mạng công nghiệp lần thứ tư (1 -\nđộng cơ hơi nước, 2 - năng lượng điện, 3 - công nghệ thông tin). AI hiện diện trong mọi lĩnh vực\ncủa đời sống con người, từ kinh tế, giáo dục, y khoa cho đến những công việc nhà, giải trí hay thậm\nchí là trong quân sự. Những ứng dụng nổi bật trong việc phát triển AI đến từ nhiều lĩnh vực để giải\nquyết nhiều vấn đề khác nhau. Nhưng những đột phá phần nhiều đến từ Deep Learning (học sâu) -\nmột mảng nhỏ đang mở rộng dần đến từng loại công việc, từ đơn giản đến phức tạp. Deep Learning\nđã giúp máy tính thực thi những việc tưởng chừng như không thể vào 15 năm trước: phân loại cả\nngàn vật thể khác nhau trong các bức ảnh, tự tạo chú thích cho ảnh, bắt chước giọng nói và chữ viết\ncủa con người, giao tiếp với con người, hay thậm chí cả sáng tác vă

In [6]:


model ="VoVanPhuc/sup-SimCSE-VietNamese-phobert-base"

embeddings = HuggingFaceEmbeddings(model_name = model)


No sentence-transformers model found with name VoVanPhuc/sup-SimCSE-VietNamese-phobert-base. Creating a new one with MEAN pooling.


In [40]:
vectorstore = FAISS.from_documents(docs, embeddings)
vectorstore.save_local("faiss_index")


In [7]:
vectorstore = FAISS.load_local("faiss_index", embeddings, allow_dangerous_deserialization=True)

# Reranking

In [8]:

def pretty_print_docs(docs):
    print(
        f"\n{'-' * 100}\n".join(
            [
                f"Document {i+1}:\n\n{d.page_content}\nMetadata: {d.metadata}"
                for i, d in enumerate(docs)
            ]
        )
    )
retriever = vectorstore.as_retriever(search_type="mmr", search_kwargs={"k": 10})

query = "định nghĩa của mạng RNN"
docs = retriever.invoke(query)
pretty_print_docs(docs)

Document 1:

văn. Đoạn đầu tiên "Mặt trời mọc ở hướng ...", ta có thể chỉ sử dụng các từ trước trong câu để đoán
là đông. Tuy nhiên, với đoạn, "Tôi là người Việt Nam. Tôi đang sống ở nước ngoài. Tôi có thể nói
trôi chảy tiếng ..." thì rõ ràng là chỉ sử dụng từ trong câu đấy hoặc câu trước là không thể dự đoán
được từ cần điền là Việt. Ta cần các thông tin từ state ở trước đó rất xa => cần long term memory
điều mà RNN không làm được => Cần một mô hình mới để giải quyết vấn đề này => Long short
term memory (LSTM) ra đời.
16.2 Mô hình LSTM
Ở state thứ t của mô hình LSTM:
•Output: ct,ht, ta gọi c là cell state, h là hidden state.
•Input: ct−1,ht−1,xt. Trong đó xtlà input ở state thứ t của model. ct−1,ht−1là output của layer
trước. h đóng vai trò khá giống như s ở RNN, trong khi c là điểm mới của LSTM.
Hình 16.2: Mô hình LSTM [25]
Metadata: {'source': 'docs/Sách Deep Learning cơ bản.pdf', 'page': 205, 'id': 285}
-------------------------------------------------------------------------------

In [9]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CohereRerank

# Create the retriever
compressor = CohereRerank(cohere_api_key='1Ld9LQLqTkOc29ABj54VRHOfyONyp4mtYIfpy7m1', top_n=5)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=retriever
)

In [10]:

compressed_docs = compression_retriever.invoke(
    query
)
print([doc.metadata["id"] for doc in compressed_docs])

[283, 249, 275, 271, 289]


In [2]:
import os

api_key = os.getenv('API_KEY')
print(api_key)

None


In [11]:

import pathlib
import textwrap

import google.generativeai as genai

from IPython.display import display
from IPython.display import Markdown


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

INFO: Tried to clean up context but failed!


In [12]:
# Or use `os.getenv('GOOGLE_API_KEY')` to fetch an environment variable.
GOOGLE_API_KEY=key

genai.configure(api_key=GOOGLE_API_KEY)

In [11]:
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-1.5-flash
models/gemini-1.5-flash-001
models/gemini-1.5-flash-latest
models/gemini-1.5-pro
models/gemini-1.5-pro-001
models/gemini-1.5-pro-latest
models/gemini-pro
models/gemini-pro-vision


In [37]:
model = genai.GenerativeModel('gemini-1.5-pro-latest')

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

def generate_response(input_text):
    doc = format_docs(compression_retriever.invoke(input_text))
#
    prompt_text = f"""
        Bạn là trợ lý cho các nhiệm vụ trả lời câu hỏi.
        Hãy trả lời câu hỏi dựa trên dữ liệu có trong đoạn ngữ cảnh
        Hãy trả lời không biết nêu không có thông tin tron đoạn ngữ cảnh

        đoạn ngữ cảnh: {doc}

        câu hỏi: {input_text}

        câu trả lời:
        """


    response = model.generate_content(prompt_text)
    return to_markdown(response.text).data


In [38]:
generate_response("hãy giải thích mạng LSTM")

'> **Mạng LSTM (Long Short-Term Memory)** là một loại mạng nơ-ron hồi quy (RNN) đặc biệt có khả năng học các phụ thuộc dài hạn trong dữ liệu tuần tự. \n> \n> **Vấn đề của RNN truyền thống:** RNN truyền thống gặp khó khăn trong việc ghi nhớ thông tin từ lâu trong chuỗi do vấn đề "vanishing gradient" (gradient tiêu biến). Khi thông tin được truyền qua nhiều bước thời gian, gradient (đạo hàm của hàm mất mát) có thể trở nên rất nhỏ, khiến mạng khó cập nhật trọng số và học các phụ thuộc dài hạn.\n> \n> **Giải pháp của LSTM:** LSTM giải quyết vấn đề này bằng cách giới thiệu một cơ chế "trạng thái ô nhớ" (cell state) đặc biệt, hoạt động như một băng chuyền thông tin. Thông tin quan trọng có thể được lưu trữ trong trạng thái ô nhớ này và được truy cập tại các bước thời gian sau đó, cho phép LSTM ghi nhớ thông tin dài hạn.\n> \n> **Cấu trúc của một ô nhớ LSTM:** Một ô nhớ LSTM bao gồm ba cổng chính:\n> \n> * **Cổng quên (forget gate):** Quyết định thông tin nào từ trạng thái ô nhớ trước đó sẽ b

In [39]:
import gradio as gr


iface = gr.Interface(generate_response, 
                     inputs="textbox",
                     outputs="textbox",
                     title="Chatbot about Deep learning",
                     description="Enter your message and the chatbot will respond accordingly.")
iface.launch()


Running on local URL:  http://127.0.0.1:7865

To create a public link, set `share=True` in `launch()`.
